In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import webcolors

In [4]:
from sklearn.cluster import MiniBatchKMeans

In [5]:
import os

In [6]:
Data_Path = os.path.join(os.getcwd(),'Data')

In [7]:
df = pd.read_csv(os.path.join(Data_Path,'Amazon Search Terms_Search Terms_US.csv'))

In [8]:
df

,Department,Search Term,Search Frequency Rank,#1 Clicked ASIN,#1 Product Title,#1 Click Share,#1 Conversion Share,#2 Clicked ASIN,#2 Product Title,#2 Click Share,#2 Conversion Share,#3 Clicked ASIN,#3 Product Title,#3 Click Share,#3 Conversion Share
0,Amazon.com,pop it,1,B08SLZ712T,"2 Packs Pop Fidget Sensory Toys, Autism Specia...",4.45%,5.21%,B08TVWR4C6,"ASONA Push Bubble Fidget Toy, Bubble Game with...",4.41%,5.22%,B094HBFB18,"Big Size Push Pop Bubble Sensory Fidget Toys,1...",4.15%,2.94%
1,Amazon.com,iphone 11 case,2,B0976787BR,"IMBZBK Shockproof Designed for iPhone 11 Case,...",15.22%,3.80%,B07WQX6V6G,Miracase Liquid Silicone Case Compatible with ...,13.81%,5.59%,B07W4FMQ5Y,"Mkeke Compatible with iPhone 11 Case, Clear Sh...",11.40%,10.69%
2,Amazon.com,bluetooth headphones,3,B095P7K3PV,"Wireless Earbuds, TWS Bluetooth Headphones, St...",11.51%,0.16%,B01HNMTCE2,iJoy Matte Finish Premium Rechargeable Wireles...,8.33%,11.09%,B07RGZ5NKS,TOZO T6 True Wireless Earbuds Bluetooth Headph...,6.57%,9.22%
3,Amazon.com,iphone 12 pro max case,4,B08RNHDV2T,Elando Crystal Clear Case Compatible with iPho...,9.66%,14.46%,B08DTLFHQF,"Mkeke Compatible with iPhone 12 Pro Max Case, ...",6.95%,9.71%,B08DY7M44F,OtterBox Commuter Series Case for iPhone 12 Pr...,6.05%,5.57%
4,Amazon.com,iphone,5,B0775MV9K2,"(Renewed) Apple iPhone 8, US Version, 64GB, Sp...",7.95%,10.92%,B07ZPKN6YR,"(Refurbished) Apple iPhone 11, US Version, 64G...",7.94%,7.42%,B01N4R20RS,"(Refurbished) Apple iPhone 7, 32GB, Black - Fu...",6.80%,13.09%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,Amazon.com,foot massager for plantar fasciitis and neurop...,"999,996",B00SMJIYNG,TheraFlow Dual Foot Massager Roller (Large). R...,29.55%,40.00%,B06WGP2CVL,"Nekteck Foot Massager with Heat, Shiatsu Heate...",29.55%,20.00%,B07F2H1NQR,"RENPHO Foot Massager Machine with Heat, Shiats...",13.64%,10.00%
999996,Amazon.com,free fire 2020,"999,997",B07ZDD1XQM,Squad Fire Survival Shooting Game,58.43%,60.00%,B078N7YVNY,"Knives Out-No rules, just fight!",30.34%,13.33%,B087XCCMPL,FPS Special Encounter Shooting: New Game 2020,8.99%,26.67%
999997,Amazon.com,boot spray deodorizer disinfectant,"999,998",B01N1375XL,"FunkAway Aerospray, 13.5 oz | Extreme Odor Eli...",31.33%,39.29%,B013TSRYUS,Natural Shoe Deodorizer Spray & Foot Odor Elim...,24.10%,21.43%,B01LMMW94Y,10-Seconds Deodorant & Disinfectant (Pack of 2...,12.05%,17.86%
999998,Amazon.com,fenix 5s plus,"999,999",B07D9FNN9W,"Garmin fenix 5s Plus, Smaller-Sized Multisport...",72.06%,90.00%,B08MR1RNXF,Garmin Fenix 5S Plus Sapphire Edition Multi-Sp...,13.97%,10.00%,B07PLNRFRD,"Garmin Fenix 5s Plus, Smaller-Sized Multisport...",9.56%,0.00%


In [9]:
ST = df['Search Term'].sample(
    n=10000,
    random_state=1
)
ST = pd.Series(ST.unique())

In [10]:
ST2 = [remove_stopwords(x)\
        .translate(str.maketrans('','',string.punctuation))\
        .translate(str.maketrans('','',string.digits))\
        for x in ST]

In [11]:
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return ' '.join(stem_sentence)
ST2 = pd.Series([stemSentence(str(x)) for x in ST2])

In [12]:
colors = list(webcolors.CSS3_NAMES_TO_HEX)
colors = [stemSentence(x) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
                                         'olive','orange','plum','salmon','tomato','wheat')]
ST2 = [' '.join([x for x in string.split() if x not in colors]) for string in ST2]

In [13]:
vectorizer_wtf = TfidfVectorizer(analyzer='word')
X_wtf = vectorizer_wtf.fit_transform(ST2)

In [14]:
result = pd.concat([ST,pd.DataFrame(X_wtf.toarray(),columns=vectorizer_wtf.get_feature_names_out())],axis=1)
result.head()

,0,aa,ab,abalon,abandon,abbey,abc,abshoo,absorb,abstract,...,zoot,zorro,zosi,zpresso,zugu,zycam,zymol,zymox,zzzquil,安博
0,queen bedset,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,norb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ring compatible door locks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,floral glue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,snap squad spinosaurus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
vectorizer_cv = CountVectorizer(analyzer='word')
X_cv = vectorizer_cv.fit_transform(ST2)

In [16]:
result = pd.concat([ST,pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names())],axis=1)
result.head()

F:\Projects_Python\Task2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,aa,ab,abalon,abandon,abbey,abc,abshoo,absorb,abstract,...,zoot,zorro,zosi,zpresso,zugu,zycam,zymol,zymox,zzzquil,安博
0,queen bedset,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,norb,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ring compatible door locks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,floral glue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,snap squad spinosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
vectorizer_ntf = TfidfVectorizer(analyzer='word',ngram_range=(1,2))
X_ntf = vectorizer_ntf.fit_transform(ST2)

In [18]:
result = pd.concat([ST,pd.DataFrame(X_ntf.toarray(),columns=vectorizer_ntf.get_feature_names_out())],axis=1)
result.head()

,0,aa,aa pack,ab,ab board,ab machin,abalon,abalon jewelri,abandon,abbey,...,zugu,zycam,zymol,zymol car,zymox,zymox plu,zzzquil,zzzquil melatonin,zzzquil night,安博
0,queen bedset,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,norb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ring compatible door locks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,floral glue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,snap squad spinosaurus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1

In [19]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_cv)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [20]:
result = pd.concat([ST,pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_cv)

In [21]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,161
1,norb,9
2,ring compatible door locks,95
3,floral glue,22
4,snap squad spinosaurus,0
...,...,...
9995,iphone 11 arm bands for exercise,159
9996,luxury bath rugs,30
9997,xlr to xlr,150
9998,14 mm male piece banger,155


In [22]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

161 queen bedset
9 norb
95 ring compatible door locks
22 floral glue
0 snap squad spinosaurus
150 standard pillowcases
127 extra long bandaids
172 yellow sheet set
8 beaded jump rope adult fitness
9 16gb ddr4 ram
112 petsafe wireless fence collar
159 omron blood pressure monitor wrist cuff
146 custom napkins
150 vasagle furniture
38 joker blu ray
159 revolution smart toaster
179 hewlett packard laptop
155 the last stand of the tin can sailors
138 portable charger power bank
127 whitey bulger
159 manimal wrist wraps
128 self watering planters and pots
154 rhino sewer hose for rv camper
136 healthy choice frozen meals
11 baby proof outlet covers
195 choco musk
134 hotel style bedding
9 hulu markers
159 bowie shirt
34 mini dresses
5 dog car divider
5 lunch box toddler
83 hanging nail polish rack
8 shower corner bench
95 rustic ring bearer box
67 breathable apple watch band
150 summer comforter
5 nba jerseys for men
5 testicuzzi for men testicle jacuzzi
83 stage 2 baby food
159 gym bands
1

150 transformers titan class
0 levis 517
29 pipe shelf
5 mommys bliss constipation ease baby
5 kong toys for puppies
2 flask for liquor for women
156 peach slices deep blemish microdarts
112 dinosaur pop it fidget toy
199 only natural pet
55 galaxy s21 ultra case with s pen holder
115 white folders
147 jogging shorts for women
0 crystal deodorant stone
155 telescopic poles
154 cup turners for tumblers starter kit
159 colored pencils erasable
108 pouf chairs
168 mint tea
146 tachometer
150 fuel line clamps
5 lunch organizer
16 27x9x14 utv tires
166 pendant light cover
55 display case lights
5 baby doppler monitor pregnancy
173 milkshake glasses
29 shittscreek
65 18x24 wood frame
27 magic wand attachments
6 wall garden
0 no one alicia keys
129 gas powered quad
2 socks women
195 yeti drinkware
156 toko cosplay
5 6 point socket set
6 garrison home hand soap
5 black tote bags
38 lost boys blu ray
195 demon slayer manga 17
51 shikai shampoo
127 belgian waffle maker
5 cleveland wedges for men

54 table saws
102 microsoft duo phone
60 thumb sleeve
169 sweet one birthday outfit girl
0 inaba
5 hair relaxer for men
5 jenni rivera books
5 organic chicken wings
146 kingdom of flesh and fire jennifer armentrout
195 battle net
156 4mm pearl beads
9 smoked tofu
34 yellow maternity dresses for baby shower
127 smarthome
190 pink dress for women sexy
34 monstera pot
179 porter robinson
55 s10 plus case
27 zapatos new balance de hombre
27 volleyball boundary lines
179 black crop tank top
51 so silver shampoo
159 bigger butt enhancement cream
155 michael jordan
146 sioris
0 dryer pedestal
57 manwich
5 game leaf blunt wraps
179 bluebird of happiness
127 sour skittles
161 down blanket queen
5 great choice dog pads
168 foam cups with lids
190 white dresses for women party
199 pulling oil
154 anker usb hub
5 screw driver kit
22 toddlers shoes
5 hair thinning shears
5 lunch storage containers
158 yellow shower curtain
146 bamboodlers
159 jem and the holograms shirt
5 red suits for men
168 plas

35 tailgate light
67 correa para apple watch series 6
159 glade automatic spray
55 iohone11promax case
30 abstract area rugs
34 mermaid bridesmaid dresses
128 water fountains indoor tabletop
125 long sleeve running shirts women
27 grill degreaser
27 mugler angel perfume
8 headband pack
6 dual car charger adapter
150 rick riordan
27 fast-pitch softball bats
146 vcds
97 salad serving bowl
159 animal crossing switch game
150 license plate bumper
5 shadow box white
150 30 bathroom vanity with sink
159 wishing well clean
29 cherry sweater
35 light blue paint
55 sumikko gurashi plush san-x licensed super cute pencil case pen pencil bag storage organizer case plush toy pen pouch pen holder (yellow)
5 newborn outfits for boys photography
154 animal dildo
127 yellow crocus
64 peaches fresh organic
93 gold baskets
115 collapsible desk for small spaces
110 kids reusable face mask boys
5 glam up easy crease line kit
93 hickory farms gift baskets
6 key rock hider
9 the flash dvd
95 ring no drill mo

179 dandelion root capsules
128 water pick teeth cleaner cordless
159 super mario odyssey - nintendo switch
5 composition notebooks wide ruled for kids
72 silicone picture frame mold
134 truck bed drawer system
112 fishing license holder waterproof
9 thinkpad t14
53 heavy moving blanket
196 belt buckles men country
115 minnie mouse tricycle
102 car phone holder cup
168 paint stand for canvas
158 shower curtain rings black
81 venus razors for women pubic hair
172 styrene sheets for models
154 50' hdmi cable
86 football toss
154 fuck biden flag
155 surface headphones 2
29 lunch meat
146 bellroy
9 this damn song
112 boob tape for breast lift
5 clip in curly hair extensions
154 bedside lamp with usb port
127 virginia ham
150 nightmare before christmas popsocket
5 infant boy outfits
155 on grief and grieving by elizabeth kubler-ross
117 dog purses for small dogs
32 batteries cr123a
29 ormus monoatomic gold
154 turtle beach ps4
22 red hots cinnamon candy
146 mene
159 nerds rope candy
54 pain

...

179 air compressor drain valve
159 adidas star wars
5 i fix it set
51 garnier dry shampoo
13 iphone 11 pro max screen protector matte
2 womens water shoes size 11
154 white fluffy pillow
195 mesh wifi 6
83 cat bathroom
8 ahegao mask
150 attack on titan
2 women nightgowns clearance
158 heavy shower curtain liner
28 sandal for women
27 texas instruments ti-30x iis
5 ky duration for men last longer
129 60w magsafe power adapter
35 turtle light
32 d batteries
154 gta ps4
159 starz app sign in
136 sony xperia 10 iii
169 skinny girl shapewear
97 gallon sprayers in lawn and garden
154 garden hose elbow
79 pool chair floats for adults heavy duty
5 hefty jumbo storage slider bags 2.5 gallon
27 infant carhartt
127 created equal
159 nintendo switch mario party
0 duoderm
148 harry potter trunk
159 o'keeffe's working hands
127 butterface
159 mesh door
29 tartarus
127 white graphic tee
195 lawn mower 4.0
159 acrylic board
100 supernatural season 15 dvd
6 gentle alarm clock
158 grey shower curtain
5 

169 newborn girl clothes summer
5 fleece pajamas baby
9 nike sb pants
97 hovercart
159 apple ipad 10.2 8th generation
155 dont stop me now
98 all dryer sheets
154 rgb laptop cooling pad
127 purge valve
2 baseball caps for women fashionable
146 ctx4
8 dr squatch soap variety pack
173 bedside carafe and glass set
155 skechers outdoor lifestyle
159 japanese hard candy
5 cinch jeans for men
127 amd rx 480
154 round bar stools
65 framed canvas
29 gtx1650
150 outside clock and thermometer for patio waterproof
150 finger skateboard
156 poly beads
99 9 inch storage bins
150 bridal jumpsuit
159 tatcha luminous dewy skin mist
179 wahl cordless senior
29 traffic racer
146 hidrate spark steel
29 nord
127 goal zero lantern
29 youth motocross goggles
159 lashes in bulk
101 sport towels for sweat
5 lange hair products
29 dumbbells 10 lbs pair
187 girl iphone 11 case
154 thrasher hat
155 cds dioxido de cloro
5 organic baby hair gel
159 love at first bite
0 slate tile
154 king kameha
150 bushnell banne

154 steno pads spiral
9 d drop
97 nenuco
127 a gentleman
67 apple watch bands 40mm women
6 bebes recien nacidos
29 insta 360onex2
150 skin tight jumpsuit
5 dewalt impact driver set
159 wwe action figure
120 led lights cheap
79 game chair with speakers
2 ahnu womens hiking boots
34 nezuko stickers
5 dog probiotic chews
155 drug test thc
134 amazon dog bed
115 mini boat
195 nyx eyeshadow primer
154 ps4 controller official
179 laptop rose gold
5 igloo dog house
29 coflex
9 america marxism
155 indoor plants live houseplants
5 henry and mudge the first book
60 black long sleeve
136 slifer the sky dragon
195 xfinity wifi extender
169 junior bridesmaid dresses for girls size 7 - 16
5 pocket puššy for men sucking
79 oversized beach chair
127 gamb
75 face wash for kids
34 ciclopirox nail lacquer
146 step bit
179 pixel 5 renewed
29 provitalize
154 double cup turner
34 mixpresso single cup coffee maker
156 stainless steel tumbler
179 oven gap guards
199 tropiclean deodorizing pet spray
5 dog butt

127 us history
168 crosswave pet pro
55 switch game holder case
27 peony perfume
154 ssx for ps4
150 butterscotch chips
97 the inv
79 tall beach chair
112 fish tape wire puller
146 jojo siwa slime
0 amish mafia
168 weathertech seat back protector
81 mens nair hair remover
29 klein flashlight
159 lego friends cubes
159 endust spray
156 bird peanuts
6 tennis tops
146 hiking daypack
0 amish murder
136 sports nutrition ready to drink protein
195 litman sthetoscopes classic iii
179 air fresherner spray
27 wii guitar
168 pill containers 2 times a day
5 dog collar tags personalized engraved
179 michelle diener
199 pug gifts for pug lovers
5 work overalls men
179 3m reflective vinyl
161 king and queen of the hood 2
5 wonderwink scrubs for men
9 encanto
195 clear puma slides
58 dunhill lighter
2 hermes belt for women
8 fidget toy pack under 10 dollars
34 corn dog sticks
159 boys shirts size 6
5 golf gloves for men
127 ruiyif
143 dark and the wiked
6 air impact wrench 1/2 inch
115 jugetes sexual

168 clear reusable cups with lids and straws
23 weber kettle grill grate
127 fabric labels for quilts
154 pillow top mattress
97 fiestaware
34 grindz grinder cleaner
136 the office a day at dunder mifflin elementary
179 vinyl stencil
6 king size mattresses 14 inch
127 leatherman rebar
115 mouse repellent for cars
166 blackout ez window covers
97 grav
169 bookbag and lunch box set for girls
168 under sink trash can with lid
0 gatorade glacier freeze
146 handycam
5 baby geniuses
95 tree rings
83 cats accessories
154 mpow usb headset
168 leisure time defender
51 love beauty and planet shampoo bar
6 classic rock cds
135 travel laptop backpack water resistant anti-theft bag with usb
159 jujubes candy
9 sourpatch
146 wrinkles schminkles
112 faux gauge earrings
127 clr
9 navy blue scrub pants
30 outdoor rug 5x8
154 terra animals
2 women footwear
65 24 x 24 frame
5 woody and buzz car hang toy
5 chevy truck model kit
195 twas the night before kindergarten
150 leather stool
159 relief band
190 b

## 2

In [23]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_wtf)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [24]:
result = pd.concat([ST,pd.DataFrame(X_wtf.toarray(),columns=vectorizer_wtf.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_wtf)

In [25]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,28
1,norb,28
2,ring compatible door locks,28
3,floral glue,149
4,snap squad spinosaurus,28
...,...,...
9995,iphone 11 arm bands for exercise,28
9996,luxury bath rugs,28
9997,xlr to xlr,28
9998,14 mm male piece banger,28


In [26]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

28 queen bedset
28 norb
28 ring compatible door locks
149 floral glue
28 snap squad spinosaurus
28 standard pillowcases
28 extra long bandaids
28 yellow sheet set
28 beaded jump rope adult fitness
28 16gb ddr4 ram
28 petsafe wireless fence collar
28 omron blood pressure monitor wrist cuff
28 custom napkins
28 vasagle furniture
28 joker blu ray
28 revolution smart toaster
28 hewlett packard laptop
28 the last stand of the tin can sailors
28 portable charger power bank
28 whitey bulger
28 manimal wrist wraps
28 self watering planters and pots
28 rhino sewer hose for rv camper
28 healthy choice frozen meals
28 baby proof outlet covers
28 choco musk
28 hotel style bedding
28 hulu markers
28 bowie shirt
28 mini dresses
28 dog car divider
63 lunch box toddler
28 hanging nail polish rack
28 shower corner bench
28 rustic ring bearer box
160 breathable apple watch band
28 summer comforter
28 nba jerseys for men
28 testicuzzi for men testicle jacuzzi
28 stage 2 baby food
28 gym bands
28 copper v

28 eliptica
28 modern witch tarot deck
28 safety bars for showers and walls
28 box car children
28 the dukes of hazzard complete series
28 round shades
28 surface piercing jewelry
28 good for u
28 spiral notebook 5x7
28 furry bean bag chair
28 charmeleon pokemon card
28 brita standard replacement filters
28 ram tailgate cap
28 women trousers
135 6ft outdoor extension cord
28 heart bowl
36 blue led light bulbs
28 minions soundtrack
28 badger foot balm
28 bathroom faucets black
28 spongebob diamond painting
28 dovo straight razor
28 titanium belly button rings
28 kindle oasis cellular
28 optical illusion shirt
28 door jammer
28 tasty bite
28 monturas de lentes
28 mini pop it pineapple
23 twin down alternative comforter
28 crocks assecories
28 mom bag for hospital
184 hat womens
28 hamburger hill
28 false teeth for missing teeth
28 pasamontañas para el sol y calor
28 beverage cooler
28 onesies for men
28 river floaties
28 removable glue
28 mens bucket hat
28 twilight books kindle
28 the f

28 back to school snacks
28 puzzle piece floor mat
28 fourth of july shirts for men
28 ginger drink
28 computer speaker with microphone
28 field hockey
28 note 10 pro xiaomi phone
28 fast car charger
28 blue area rugs 8x10
28 6-9 month girl clothes
28 waterlox original sealer finish
28 irobot movie
28 polygel nail kit with uv lamp starter kit
28 etrack
28 electric pellet smoker
28 water bottle running
28 kindergarten cop dvd
28 juicy drop taffy
28 zebco fishing reels
28 kurig pods
28 benadryl itch stick
26 garmin 45
28 hugo boss polo shirt men
28 hdmi cables 2 pack
28 bookends
28 1776 the musical
28 air doctor replacement filters
28 royal canin medium adult dog food
28 screen protector iphone 6
28 boys in the boat book
28 hallmark cards assortment box
28 deodorant bulk
28 large duffle
28 marbles bulk
28 rk87
28 specialty knives
28 harry potter halloween decorations
28 rice a roni
28 mini cast iron skillet
28 cute shelves
28 gluten free ladyfingers
28 anderson hitch
28 flower bedding
28

28 dallas cowboy cheerleader costume
28 kabar spork
28 1ml syringe without needle
28 microtouch
28 starbucks italian roast k cups
28 adzuki beans
28 oxo frying pan
28 curtains for bedroom
28 womens dressy tops
28 letterpress machine
23 ivory comforter set queen
28 white fringe top
28 lash cleansing brush
28 cassia oil
28 red stove knobs
28 solar lanterns lism
28 round here
28 toilet with sink on top
28 at last
28 trinidad flag
28 mediterranean life
28 small animal hammock
63 bento box adult lunch box with bag
28 mama said
28 byootique
28 white patio furniture
28 braun razor cleaner refills
28 sriracha powder
28 brushless motor and esc 1/10 scale
28 forehead light
28 wood patio table
28 sega
28 lowrance fishfinder
28 cute one piece swimsuit for women
28 rog strix g15 advantage edition
28 cocomelon 1st birthday party supplies
28 teen girl backpack with lunch box
28 stainless steel wire brush
28 bosley
28 the only astrology book you'll ever need
28 groomsmen gifts for wedding
28 portrait 

28 personalized dog collar
28 tommy bahama hat
59 11 pro max iphone case
28 plastic forks and knives
130 16 birthday decorations
28 weiman jewelry cleaner
28 sleeping earbuds
28 monopoly deal
151 toddler sheets for toddler bed girl
28 ear cleaner for dogs
117 pencil roll up case
28 office music
28 ice cream for dogs
28 botas de trabajo para hombre wolverine
28 copper fit ankle brace for men
28 thai green curry paste
28 padded bralettes for women
28 glitter sticker paper
28 portable air conditioner cover
28 face exfoliant
28 mountain man romance
28 melissa and doug doll
28 white beaded necklace
28 kitchen carpet
28 mens brooks ghost 14
28 resin filler
28 decorative clock
28 broken jenny lawson
28 cr2032 3v lithium battery 2 pack
28 meat cutter machine
28 rose gold napkin rings
28 virtual gift card
28 genesis license plate frame
28 displayport to hdmi 4k
179 jess michaels
28 school bag kawaii
28 wall plug in wax warmer
28 dell computer charger
28 orange dog leash
28 donut float
28 stihl 

...

28 alfombras de cocina
149 floral robe
28 samsung kids tablet
28 metal pill organizer
28 tuf gaming
28 mukluks slippers for women
28 kurdene bluetooth earbuds
28 classroom rug
28 queen bed platform with storage
28 foxmind
28 70s disco
28 neutral shower curtain
28 tazer gun
28 bedhead after party
28 organic frozen fruit
28 helix hoop earring
28 nessie ladle
28 the iron mask
28 yankee candle essential oil
28 macaron boxes for 6
28 i spy movie prime
28 power metal-cutting & chop saws
28 eyelash tweezers bulk
28 snail food
28 inflatable paint booth for cars
28 murakami
28 curing pills
28 gucci t shirts for men
28 studio beats buds case
28 piano keyboard
28 van organizer
28 michael stars dress
28 drawer organizers kitchen
28 dog blankets for medium dogs
28 kids doctor kits 3-5 years
28 glass l shaped desk
28 trim router
28 balcony lights outdoor waterproof
28 white gold wedding band
28 55 inch monitor
28 mental game of trading
28 kona ice machine
28 brazilian waxing kit
28 outdoor dining se

28 dome tent stand up
28 purple fishing pole
28 c5 corvette cup holder
28 samsung tv base stand
28 narrow walkers for seniors
28 blue metallic balloons
28 bunk bed canopy for top bunk
28 side table round
28 candle vanilla
28 diapers and wipes
28 ponds face wash
28 canning jars 8 oz
28 moped for teens
28 trolls dolls
28 top loading cat carrier
28 dont cha
28 guitar stand acoustic
28 silver cross pendant for men
28 white fireplace
28 double ended makeup brush
28 sport clothes for women
28 gmc denali accessories
28 single ply toilet paper
28 gel nail uv light professional
28 pillow travel bag
28 emborady machine
28 starman movie
28 reading comprehension grade 6
28 injectable b12
28 glucosamine chondroitin for dogs
28 150 qt cooler
28 spring valley hair skin and nails vitamins
28 worm threader
28 ranch hand
28 large d20
28 michelle branch
28 leigon
28 aluminum cups disposable
28 bluetooth mechanical keyboard
28 glock 18c bb gun full auto
28 newborn girl clothes summer
28 fleece pajamas bab

28 ciclopirox nail lacquer
28 step bit
28 pixel 5 renewed
28 provitalize
28 double cup turner
28 mixpresso single cup coffee maker
28 stainless steel tumbler
28 oven gap guards
28 tropiclean deodorizing pet spray
28 dog butt hooks
28 the betty broderick story
28 eyebrow wigs for women
28 mouse pad wrist support
28 silk pajama
28 famous stars and straps
28 60 quart cooler
28 basket set
28 vin glace
28 speed out screw extractor set
28 tension rod support
28 prayer shawl for women christian
28 little einsteins prime video
28 reebok backpack
28 pop it owl
28 hills
28 womens dressy romper
28 spinning shoes
28 french bulldog toys
28 ice cream maker'
166 oxygenics evolution rain shower head
28 comcast approved routers
28 moen aerator
28 yeti 20 oz tumbler handle
28 change counter
28 alesis surge mesh drum kit
28 cuff earrings for women no piercing
28 gold table confetti
28 venom party supplies
28 leaf canopy
28 plus size high waisted jeans for women
28 pet cave
137 battery for kids jeep
28 98

28 labtops, hp touchscreen
28 the silk road
59 iphone 11 pro max case for women
28 beauty stickers
28 long handled shower scrubber
28 xtar
199 backpacks black
28 usb extension
28 molar teether
28 kiss dip powder
28 rv furnace vent screen
28 gardening seeds
98 funwater 11 inflatable stand up paddle board
171 wireless car mount charger
28 cheese american
28 make up tables for women with lighted mirror
28 crochet hook grips
28 alzheimers gps watch
28 hitch bike rack 2
28 daddy here comes mommy wedding sign
28 honest sanitizer spray
28 girls terry cloth swim cover up
28 table top protector
28 balance complex for women
28 deshedding shampoo for dogs
28 earth origins shoes women
28 pokemon mug
28 mysteries of the abandoned
28 thin magnets
86 baby heart rate monitor for pregnancy
28 campfire color flames
28 large travel toiletry bag
28 home networking panel
28 breastmilk ice pack
28 leaves room decor
30 electric blanket king size
28 clear pen holder
28 garden saw
28 compact mirror magnifying


28 tea tree deodorant
28 baby shark cupcake stand
59 iphone 11 case girls
28 mesitas de noche
28 non lethal weapons
28 circut machine 2
28 toilet seat replacement
28 empty eyeshadow palette
28 bread lame
28 chinese food containers
28 the days of noah part 1-4
28 size 7 rings for women
28 hidden camera detectors
28 elephant walk
28 bike locks
28 photoelectric switch
28 pool sanitizer
63 bento lunch
28 tijan kindle books
28 gods
28 raise your voice
28 bradford guardian 3
28 patanjali aloe vera gel for face
28 gevalia coffee k cups
28 dried grass
28 cruella 2021
28 clipper guard holder
28 caldigit element hub
28 real little
28 geometric peel and stick wallpaper
28 junction box plastic
28 barbie looks doll
28 cottage cheese whole milk
28 living room decor clearance
28 foam slides for women
28 etg test
28 wifi 6 card
28 civil war marvel
28 krydex
28 kebab maker
28 gold 8x10 picture frames
28 ameda mya breast pump accessories
28 red queen
28 cashmere woods glade refills
28 card games book
28

## 3

In [27]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_ntf)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [28]:
result = pd.concat([ST,pd.DataFrame(X_ntf.toarray(),columns=vectorizer_ntf.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_ntf)

In [29]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,17
1,norb,17
2,ring compatible door locks,17
3,floral glue,17
4,snap squad spinosaurus,17
...,...,...
9995,iphone 11 arm bands for exercise,17
9996,luxury bath rugs,17
9997,xlr to xlr,17
9998,14 mm male piece banger,17


In [30]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

17 queen bedset
17 norb
17 ring compatible door locks
17 floral glue
17 snap squad spinosaurus
17 standard pillowcases
17 extra long bandaids
17 yellow sheet set
17 beaded jump rope adult fitness
17 16gb ddr4 ram
140 petsafe wireless fence collar
17 omron blood pressure monitor wrist cuff
17 custom napkins
17 vasagle furniture
17 joker blu ray
17 revolution smart toaster
17 hewlett packard laptop
17 the last stand of the tin can sailors
17 portable charger power bank
17 whitey bulger
17 manimal wrist wraps
17 self watering planters and pots
17 rhino sewer hose for rv camper
17 healthy choice frozen meals
17 baby proof outlet covers
17 choco musk
17 hotel style bedding
17 hulu markers
17 bowie shirt
17 mini dresses
17 dog car divider
17 lunch box toddler
17 hanging nail polish rack
17 shower corner bench
17 rustic ring bearer box
17 breathable apple watch band
17 summer comforter
17 nba jerseys for men
17 testicuzzi for men testicle jacuzzi
17 stage 2 baby food
17 gym bands
17 copper vi

17 box car children
17 the dukes of hazzard complete series
17 round shades
17 surface piercing jewelry
17 good for u
17 spiral notebook 5x7
17 furry bean bag chair
17 charmeleon pokemon card
17 brita standard replacement filters
17 ram tailgate cap
17 women trousers
17 6ft outdoor extension cord
17 heart bowl
17 blue led light bulbs
17 minions soundtrack
17 badger foot balm
17 bathroom faucets black
17 spongebob diamond painting
17 dovo straight razor
17 titanium belly button rings
17 kindle oasis cellular
17 optical illusion shirt
17 door jammer
17 tasty bite
17 monturas de lentes
17 mini pop it pineapple
17 twin down alternative comforter
17 crocks assecories
17 mom bag for hospital
17 hat womens
17 hamburger hill
17 false teeth for missing teeth
17 pasamontañas para el sol y calor
17 beverage cooler
17 onesies for men
17 river floaties
17 removable glue
17 mens bucket hat
87 twilight books kindle
17 the five dysfunctions of a team
17 disney luggage for adults
17 orfeld vacuum
17 wi

17 dude bomb
17 refrigerator light bulb led
17 guitar hero xbox one
17 chingy
17 color printer paper 8.5x11
17 90s hip hop greatest hits
17 resin gold flakes
17 the honest company shampoo
17 knife glove
17 closet bar bracket
17 dog name tag
17 carhartt work pants for men
17 pleystation5
17 zipper door
17 french cooking
17 expo dry erase
17 guantes para moto
120 always overnight pads
17 baby combat boots
17 platform heels for women exotic
17 file folder games
17 agree shampoo and conditioner
17 exercise rope
17 running pants men
17 under armour freedom hat
17 18 inch curtain rods
17 men's lounge shorts
17 tiki torch canister
17 polygel nail kit with led lamp
17 andrew weil
17 sugar free cookies for diabetics
17 g29 logitech racing wheel
17 medellin j balvin
17 brown fur slides
17 hp 4155e all in one printer
17 back to school snacks
17 puzzle piece floor mat
17 fourth of july shirts for men
17 ginger drink
17 computer speaker with microphone
17 field hockey
17 note 10 pro xiaomi phone
17

17 microtouch
17 starbucks italian roast k cups
17 adzuki beans
17 oxo frying pan
17 curtains for bedroom
17 womens dressy tops
17 letterpress machine
17 ivory comforter set queen
17 white fringe top
17 lash cleansing brush
17 cassia oil
17 red stove knobs
17 solar lanterns lism
17 round here
17 toilet with sink on top
17 at last
17 trinidad flag
17 mediterranean life
17 small animal hammock
17 bento box adult lunch box with bag
17 mama said
17 byootique
17 white patio furniture
17 braun razor cleaner refills
17 sriracha powder
17 brushless motor and esc 1/10 scale
17 forehead light
17 wood patio table
17 sega
17 lowrance fishfinder
17 cute one piece swimsuit for women
17 rog strix g15 advantage edition
17 cocomelon 1st birthday party supplies
17 teen girl backpack with lunch box
17 stainless steel wire brush
17 bosley
17 the only astrology book you'll ever need
17 groomsmen gifts for wedding
17 portrait of the artist as a young man
17 dvd player car
17 mun
17 diapers size 2 pampers sw

17 ice cream for dogs
17 botas de trabajo para hombre wolverine
17 copper fit ankle brace for men
17 thai green curry paste
17 padded bralettes for women
17 glitter sticker paper
17 portable air conditioner cover
17 face exfoliant
17 mountain man romance
17 melissa and doug doll
17 white beaded necklace
17 kitchen carpet
17 mens brooks ghost 14
17 resin filler
17 decorative clock
17 broken jenny lawson
17 cr2032 3v lithium battery 2 pack
17 meat cutter machine
17 rose gold napkin rings
17 virtual gift card
17 genesis license plate frame
17 displayport to hdmi 4k
17 jess michaels
17 school bag kawaii
17 wall plug in wax warmer
17 dell computer charger
17 orange dog leash
17 donut float
17 stihl 2 cycle oil
17 nvg
17 garmin venu band
17 thick guitar pick
17 belly bands for postpartum women
17 leather satchel for men
17 long romper
17 skinny highlighters
17 travel pod apple watch charger
17 jordan socks for men
17 leg brace for dogs back leg
17 bringing up baby criterion blu ray
17 jessic

...

17 policosanol 20mg
17 electronic fly swatter zapper
17 flags and poles
17 humann
17 cleaning gloves xl
17 tribal brand clothing
17 evenflo safemax car seat base
17 hippie sandals
17 luxe bidet
17 cute car air freshener
17 tsa approved clear toiletry bag
17 blueface
17 birchwood casey super blue
17 dandelion weeder
17 honey dispenser no drip glass with stand
17 turkey tail mushrooms
17 funny golf shirt
47 iphone 12 pro case with card holder
17 dr brown size 2 nipples
17 thats not my puppy
17 kitchen shelves organizers and storage
17 three keys
17 cherry dresser
17 brain puzzles for adults
17 chevignon
17 credit card skin
17 gold toddler sandals
17 dr. browns level 3 nipples
17 ifergoo clay
17 smoothie bottles reusable
17 the autopsy of jane doe
17 headset for switch
17 tiny orb necklace
17 car bin
17 apple airpods charging case
17 logitech g213 prodigy
17 heavy duty extension cord 25 ft
17 hoverboard razor
17 haf-cin
17 fidget spinner pen
17 styx cd
17 lawn bug spray
17 pure ammonia
17

17 ipad air 3 keyboard
17 spring loaded access panel
17 nail envy nail strengthener
17 sequin wall
17 clocks for living room decor large
17 plus size navy blue dress for women
17 rejas para perros
17 planter pot
17 guar
17 natori sports bras for women
17 hostiles movie
17 youre the one bath and body works
17 giant candle
17 heat transfer tape for heat press
17 waterproof queen mattress pad
17 electric bike kit
17 citronella wax melts
17 popcorn cupcake wrappers
17 labor saving devices
17 hiking supplies
17 fall confetti
158 red shower curtain hooks
17 xingqijia
17 boho macrame wall hanging
17 coral napkins
17 gallon water bottle with handle
17 wild cat fortnite pack code
17 boat tubes and towables 3 person
17 silicone scar sheet
17 white fur throw
17 aero precision scope mount 30mm
17 cerave facial moisturizing lotion am spf 30
17 sigma 16mm 1.4 canon ef-m
17 lip plumper gloss
17 canon dslr camera
17 traxxas slash shocks
17 blues brothers movie
17 electronic face mask
17 wedding gift
1

17 play money
17 verizon remote control fios
17 club dresses for curvy women
17 semi formal dress
17 half gallon stainless steel water bottle
17 centrum for men
17 cute dinosaur
17 led lights for shop
17 decked
17 wood name sign
17 tuna helper
17 book holder for reading in bed
17 small air conditioner window unit
17 gpca carabiner
17 hand soap refills antibacterial
17 cushionaire shoes
17 mr. smith goes to washington
17 bed wedge pillow
17 hanging flower garland
17 spongebob toy
17 stacking shelf
17 winch fairlead
17 fluke anchor
17 downy eco box fabric softener
17 the truants
17 fiskars scissors 5 inch
17 tv wall mount 50 in
17 sparkly black dress
17 oil free face sunscreen
17 aviator nation sweatpants
17 russian shirt
17 inflatable beach lounger
17 storybots
17 zero breeze portable air conditioner
17 dorm room desk
17 waterproof tablecloth rectangle
17 candle ear wax removal kit
17 neon apple watch band
17 thermo pro thermometers
17 grey crocs
17 universal diffuser
17 mega fidget pac

17 tactical gear for men
17 thin condoms
17 white bedspread
17 pillow block bearing
17 kenmore electric range drip pans
17 instant pot nova
17 totoro plushie
17 12 x 18 white construction paper
17 artifical flowers
17 gummy probiotics
17 broken vows
17 leopard print masks
17 hi pro pac extremely damaged hair intense protein treatment
17 universal rocks
17 magic the gathering land cards
17 2022-2023 monthly planner
17 65 inch smart tv 4k samsung
17 aluminum screen
17 mcculloch
17 cw
17 aveeno kids body wash
17 devil
17 boho pillow cases
17 tarot card deck
17 365 multivitamin
17 jeep soft top straps
17 celery juice book
17 small buffer polisher
17 xs glass screen protector
17 halloween baby blanket
17 airpod silicone ear tips
17 maid costume
17 optix 55
17 ii inin womens deep v neck casual dress backless
17 allegra k dresses for women
17 sugar free black licorice
17 bag holder for car
17 skybrite star projector
17 trump 4th of july shirt
17 loungfly
17 kobe socks
17 dog splash pool
17 gr

17 adidas forum mid
17 long mirrors for wall
17 fds
17 6inch cake round pan
17 gavita pro 1700e led
17 womens western dress
17 clips for outdoor string lights
17 office chair with arms
17 sofa bed wood
17 laugh
17 pregnenolone 30mg
17 tidal 55 aquarium filter
17 cast iron holder handle
17 skechers beach casuals
17 zenitsu shoes
17 lumipets
17 shadow box 16x20
17 women scrub set
17 farmhouse planter
17 humankind
17 shell ring
17 resin craft kit
17 cosmetic spatulas
17 high tech pet collar
17 angies boom chicka pop
17 syrup dispenser for coffee bar
17 friday night funkin shirt
17 perrier strawberry sparkling water
17 nosalt
17 chef knife set with bag
17 toys for boys age 2
17 day beds for kids
17 zonko tablet
17 franklin sensors
17 full length mirror
17 solar recovery
17 electrical socket cover
17 sebastian yatra
17 aprons for men with pockets
17 vienna sausages
17 decoder ring
17 wine book
17 sawzall dewalt
17 msi gaming mouse
17 patio table cover with umbrella hole
17 paper folding mac